In [25]:
import argparse
import csv
import datetime
import itertools
import logging
import multiprocessing
import os
import sys
import time
import traci
from collections import Counter
import jsonpickle

from files_to_create_zones.config import Config
from files_to_create_zones.data import Data
import files_to_create_zones.emissions as emissions
from utils.model import Emission

In [2]:
simulation_dir = 'F:/SUMO/sumo-grid/'

In [3]:
for f in os.listdir(simulation_dir):
    if f.endswith('.sumocfg'):
        _SUMOCFG = os.path.join(simulation_dir, f)
        
sumo_binary = os.path.join(os.environ['SUMO_HOME'], 'bin', 'sumo-gui')
sumo_cmd = [sumo_binary, "-c", _SUMOCFG]

In [ ]:
## code for divide the grid

# traci.start(sumo_cmd)
# areas_number = 2
# dump_name = 'new_dump_2'
# if not os.path.isfile(f'{simulation_dir}/dump/{dump_name}.json'):
#     start = time.perf_counter()
#     data = Data(dump_name, traci.simulation.getNetBoundary(), areas_number, simulation_dir)
#     data.init_grid()
#     data.add_data_to_areas() 
#     data.save()
    
#     loading_time = round(time.perf_counter() - start, 2)
#     print(f'Data loaded ({loading_time}s)')
#     print(f'Dump {dump_name} created')
# else:
#     print(f'Dump with name {dump_name} already exists')
    
# traci.close(False) 

In [9]:
!pip install "stable_baselines3[extra]>=2.0.0a9"

  Using cached stable_baselines3-2.3.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.2.2-cp38-cp38-win_amd64.whl.metadata (26 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached matplotlib-3.7.5-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached pygame-2.5.2-cp38-cp38-win_amd64.whl.metadata (13 kB)
  Using cached tensorboard-2.14.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached Shimmy-1.3.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pillow-10.3.0-cp38-cp38-win_amd64.whl.metadata (9.4 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached click-8.1.7-py3-none-any.whl.

In [8]:
traci.start(sumo_cmd)
step = 0
action = 30
while step < 200:
    traci.simulationStep()

    vehicles = emissions.get_all_vehicles()


    number_of_lanes = traci.lane.getIDCount()
    print(number_of_lanes)
    count_vehc = 0
    lane_length_sum = 0
    mean_speed_sum = 0
    occupancy_sum = 0
    waiting_time_sum = 0
    travel_time_sum = 0 
    halting_sum = 0
    vehicle_length_sum = 0

    lanes = traci.lane.getIDList()
    for lane in lanes:
        traci.lane.setMaxSpeed(lane, action)
        # print(traci.lane.getMaxSpeed(lane))
        count_vehc += traci.lane.getLastStepVehicleNumber(lane) #int
        lane_length_sum += traci.lane.getLength(lane) #m
        mean_speed_sum += traci.lane.getLastStepMeanSpeed(lane) #m/s
        # vehicle_length_sum += traci.lane.getLastStepLength(lane) #m
        occupancy_sum += traci.lane.getLastStepOccupancy(lane) #%
        waiting_time_sum += traci.lane.getWaitingTime(lane) #s
        travel_time_sum += traci.lane.getTraveltime(lane) #s
        halting_sum += traci.lane.getLastStepHaltingNumber(lane) #int

    mean_speed = mean_speed_sum/number_of_lanes
    occupancy = occupancy_sum/number_of_lanes
    # occupancy_cal = vehicle_length_sum/lane_length_sum
    lane_density = count_vehc/lane_length_sum
    waiting_time = waiting_time_sum/number_of_lanes
    travel_time = travel_time_sum/number_of_lanes

    print(f'count of vehicles: {count_vehc}')
    print(f'mean speed = {mean_speed}')
    print(f'occupancy = {occupancy}')
    # print(f'occupancy cal = {occupancy_cal}')
    print(f'lane density = {lane_density}')
    print(f'waiting_time = {waiting_time}')
    print(f'travel_time = {travel_time}')
    print(f'halting_sum = {halting_sum}')


    speed = 0
    # for vehicle in vehicles:
    #     speed += traci.vehicle.getSpeed(vehicle)

    # print(f'mean speed vehc = {speed/len(vehicles)}')

    print(len(vehicles))
    veh_types = []

    # total_emissions = Emission()
    

    # for vehicle in vehicles:
    #     veh_types.append(vehicle.type)
    #     total_emissions += vehicle.emissions
    step += 1
    # print(total_emissions)

    # total_cars = Counter(veh_types)['normal'] + Counter(veh_types)['sporty']
    # total_trucks = Counter(veh_types)['coach'] + Counter(veh_types)['trailer']

    # print(total_cars)
    # print(total_trucks)


    # total = total_emissions.co2 + total_emissions.co + total_emissions.nox + total_emissions.hc + total_emissions.pmx
    
    # print(total)

    

traci.close(False)

282
count of vehicles: 10
mean speed = 28.93617021276596
occupancy = 0.0003624084125034618
lane density = 0.00016435549963085788
waiting_time = 0.0
travel_time = 35467.606491725746
halting_sum = 10
10
282
count of vehicles: 10
mean speed = 28.999472019791764
occupancy = 0.0003624084125034618
lane density = 0.00016435549963085788
waiting_time = 0.0
travel_time = 16.81082872786083
halting_sum = 0
10
282
count of vehicles: 10
mean speed = 29.07852350724687
occupancy = 0.0003624084125034618
lane density = 0.00016435549963085788
waiting_time = 0.0
travel_time = 10.970489838305586
halting_sum = 0
10
282
count of vehicles: 20
mean speed = 29.04164187882545
occupancy = 0.0007248168250069237
lane density = 0.00032871099926171576
waiting_time = 0.0
travel_time = 12.481177720942489
halting_sum = 10
20
282
count of vehicles: 20
mean speed = 29.12267339340857
occupancy = 0.0007248168250069237
lane density = 0.00032871099926171576
waiting_time = 0.0
travel_time = 9.92432492366068
halting_sum = 0
20


In [10]:
!pip install gym

  Using cached gym-0.26.2-py3-none-any.whl
  Using cached gym_notices-0.0.8-py3-none-any.whl.metadata (1.0 kB)
Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)


In [26]:
traci.close(False)

In [52]:
ee = [0, 0, 0, 0, 0]

In [54]:
import numpy as np
np.array(ee)

array([0, 0, 0, 0, 0])

In [11]:
speeds = [40, 50, 60, 70, 80, 90, 100]

In [51]:
traci.getVersion() 

FatalTraCIError: Not connected.

In [13]:
speeds[0]

40

In [1]:
from gymnasium import spaces

In [2]:
action_space = spaces.MultiDiscrete([7]) 

In [21]:
et = action_space.sample()

In [22]:
et

array([-0.6092096], dtype=float32)

In [25]:
action[0]

0.7874923

In [5]:
action

array([0], dtype=int64)

In [70]:
action = [5]

In [9]:
%pip install parse

  Using cached parse-1.20.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached parse-1.20.1-py2.py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from TrafficEnv import SpeedLimitEnv


world = SpeedLimitEnv()

In [28]:
world.reset()

[0, 0, 14.615531914893666, 0.0, 0.0]


(array([ 0.      ,  0.      , 14.615532,  0.      ,  0.      ],
       dtype=float32),
 {})

In [12]:
et

array([1], dtype=int64)

In [29]:
obs, reward, done, info = world.step(et)

TypeError: 'int' object is not callable

In [20]:
action_space = spaces.Box(low=-1, high=1,shape=(1,),dtype="float32")